# Historical weather extraction

At SetMagic Productions, we have identified the need to have a historic of data for each of our candidate cities to film in. We want to have the data to work with it and perform our in-house analysis.

For that matter, we have chosen to scrape historical weather data from the webpage [Wunderground](https://www.wunderground.com). In this notebook, we detail the process followed to scrape, process and save the historic data from 5 random cities, from the beginning of the year up until now.

#### 0. Imports

Import necessary libraries.

In [1]:
# Importamos las librerías que necesitamos

# Librerías de extracción de datos
# -----------------------------------------------------------------------

# Importaciones:
# Beautifulsoup
from bs4 import BeautifulSoup

# Requests
import requests

import pandas as pd
import numpy as np

from time import sleep

# Importar librerías para automatización de navegadores web con Selenium
# -----------------------------------------------------------------------
from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException # Excepciones comunes de selenium que nos podemos encontrar 

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my-geopy-app")
import random
import re
import datetime

# import suppor functions
import sys 
sys.path.append("..")
from src.support_historical_weather import random_cities, get_city_table

### 1. Wunderground scraping process

### 1.1 Explore the page to identify scraping link

After an initial exploration of the website, have already identified the link to most easily automate the information scraping from wunderground below. With it, it is possible to use selenium to navigate to the bottom of each city's page and extract the text for each city's table:

In [2]:
table_link = "https://www.wunderground.com/dashboard/pws/IVILLA665/table/2024-09-01/2024-09-31/monthly"

By iterating over the city code, the years and the months, the historic of data for any date and city can be extracted:

"https://www.wunderground.com/dashboard/pws/{city_code}/table/{year}-{month}-01/{year}-{month}-31/monthly"

### 1.2 Scraping process outline

However, to use that url it is necessary to have that code. Besides, the business case instructs that 5 random cities could be needed for extraction, therefore it is necessary to automate the code extraction too. Thus the process outline would be:

1. Randomly choose a city from the previous' days coordinates file.
2. Scrape from a base link Wunderground to get city codes for cities without itand add the codes to the cities file.
3. With codes in hand, scrape Wunderground again, using the final link to scrape historic weather tables.
4. Process the obtained DataFrame
5. Save the processed information

## 2. Scraping

### 2.1 Load the city coordinates DataFrame

In [5]:
city_coordinates = pd.read_csv("../../lab_1_lunes/data/city_coordinates.csv", index_col=0)

# Add a code_wunder column to save the extracted codes
city_coordinates["code_wunder"] = ""
city_coordinates

,city,latitude,longitude,code_wunder
0,acebeda-la,41.086958,-3.624399,
1,ajalvir,40.534230,-3.480782,
2,alameda-del-valle,40.917718,-3.843822,
3,alamo-el,40.230730,-3.990589,
4,alcala-de-henares,40.481840,-3.364497,
...,...,...,...,...
174,villar-del-olmo,40.336447,-3.235562,
175,villarejo-de-salvanes,40.168358,-3.273876,
176,villaviciosa-de-odon,40.357379,-3.900233,
177,villavieja-del-lozoya,41.006362,-3.671629,


### 2.2 Randomly choose cities and scrape their codes

Using the support functions in the ``support_historical_weather.py`` file in the src/ folder, codes are extracted for each city and saved into the city coordinates file.

```python
def random_cities(n_cities, cities_df):

    randoms = cities_df.sample(n=n_cities)
    code_list = list()
    for idx,random_city in randoms.iterrows():
        if random_city["code_wunder"] == "":
            random_city["code_wunder"] = get_city_code(random_city["city"])
            cities_df.loc[cities_df["city"] == random_city["city"] ,"code_wunder"] = random_city["code_wunder"]

        code_list.append(random_city["code_wunder"])
    
    return code_list

def get_city_code(city_name):
    link = f"https://www.wunderground.com/weather/es/{city_name}"

    driver = webdriver.Chrome()
    driver.get(link)

    driver.maximize_window()

    handle_cookies(driver)

    #display table on the page
    sleep(5)
    try:
        code_link = driver.find_element("css selector","#inner-content > div.region-content-top > lib-city-header > div:nth-child(1) > div > div > a.station-name").get_attribute("href")
        driver.get(code_link)
    except:
        print("Url to capture the city_code not well got.")
        
    sleep(3)
    code = driver.find_element("css selector", "#inner-content > div.region-content-top > app-dashboard-header > div.dashboard__header.small-12.ng-star-inserted > div > div.heading > h1")
    code = code.text.split(" - ")[-1]
    return code
```

In [7]:
city_codes = random_cities(5, city_coordinates)
city_codes

['ICAMAR23', 'IALCOB23', 'ISANTA956', 'IVILLA660', 'IVILLA1123']

In [8]:
city_coordinates.to_csv("../../lab_1_lunes/data/city_coordinates.csv")

### 2.3 Scrape the historical weather tables with the codes

Using the support functions in the ``support_historical_weather.py`` file in the src/ folder, the historical weather tables are extracted into an already formatted DataFrame and concatenated into one.

In [9]:
months_to_scrape = [1,2,3,4,5,6,7,8,9]
df_final = pd.DataFrame()

for month in months_to_scrape:
    for city_code in city_codes:
        try:
            df = get_city_table(city_code,2024,month)
        except:
            print(f"There was a problem scraping the citiy code {city_code} for the month {month}.")
        
        df_final = pd.concat([df_final,df],axis=0)

df_final

There was a problem scraping the citiy code ISANTA956 for the month 4.
There was a problem scraping the citiy code IVILLA1123 for the month 7.
There was a problem scraping the citiy code ICAMAR23 for the month 8.
There was a problem scraping the citiy code IALCOB23 for the month 8.
There was a problem scraping the citiy code ISANTA956 for the month 8.
There was a problem scraping the citiy code IVILLA660 for the month 8.
There was a problem scraping the citiy code IVILLA1123 for the month 8.
There was a problem scraping the citiy code ICAMAR23 for the month 9.
There was a problem scraping the citiy code IALCOB23 for the month 9.
There was a problem scraping the citiy code ISANTA956 for the month 9.
There was a problem scraping the citiy code IVILLA660 for the month 9.
There was a problem scraping the citiy code IVILLA1123 for the month 9.


,Date,Temp High ºF,Temp Avg ºF,Temp Low ºF,Dew High ºF,Dew Avg ºF,Dew Low ºF,Humid High %,Humid Avg %,Humid Low %,Speed High mph,Speed Avg mph,Speed Low mph,Pressure High in,Pressure Low in,Precip Sum in,city_code
2,2024-01-01,49.3,39.5,33.3,40.8,37.4,32.0,99,93,66,7.6,1.2,0.0,30.22,30.09,0.00,ICAMAR23
3,2024-01-02,44.6,39.5,32.9,44.2,38.5,32.4,99,97,86,5.4,0.8,0.0,30.23,30.12,0.00,ICAMAR23
4,2024-01-03,54.9,50.0,44.2,53.6,49.5,43.9,99,98,95,13.9,3.2,0.0,30.15,30.06,0.21,ICAMAR23
5,2024-01-04,50.9,48.1,44.6,50.4,47.7,44.2,99,99,95,6.7,0.9,0.0,30.14,29.84,0.22,ICAMAR23
6,2024-01-05,47.8,44.4,39.6,46.8,38.7,32.0,99,81,57,22.1,4.2,0.0,29.95,29.76,0.03,ICAMAR23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,2024-07-27,100.0,84.5,68.9,68.4,64.1,59.5,84,53,27,10.7,2.1,0.0,29.90,29.77,0.00,IVILLA660
29,2024-07-28,103.3,86.8,68.4,67.3,61.7,55.6,82,46,22,10.7,1.6,0.0,29.93,29.81,0.00,IVILLA660
30,2024-07-29,96.6,85.9,73.9,71.8,62.2,52.2,72,47,29,25.1,2.9,0.0,30.00,29.79,0.00,IVILLA660
31,2024-07-30,102.0,86.5,71.2,71.4,66.0,55.9,89,53,29,10.5,1.9,0.0,29.95,29.84,0.00,IVILLA660


### 2.4 Process the obtained information

The Dataframe is directly formatted during the scraping process to have each day's and city weather information. Consult `support_historical_weather.py` for more details.

```python
def scrape_table(driver, city_code):
    # get element with the table rows inside
    rows = driver.find_element("css selector", "#main-page-content > div > div > div > lib-history > div.history-tabs > lib-history-table > div > div").text.split("\n")

    # remove special characters and split into cells
    df_table = pd.Series([row.replace("°F","").replace("%","").replace("mph","").replace("in","") for row in rows])
    df_table = df_table.str.split(expand=True)

    # format columns
    df_table.columns = df_table.iloc[1]
    df_table = df_table[2:]
    df_table["Date"] = pd.to_datetime(df_table["Date"])
    df_table.columns = ['Date', 'Temp High ºF', 'Temp Avg ºF', 'Temp Low ºF', 'Dew High ºF', 'Dew Avg ºF', 'Dew Low ºF', 'Humid High %', 'Humid Avg %',
        'Humid Low %', 'Speed High mph', 'Speed Avg mph', 'Speed Low mph', 'Pressure High in', 'Pressure Low in', 'Precip Sum in']
    
    # add city code
    df_table["city_code"] = city_code
    
    return df_table
```

In [10]:
df_final

,Date,Temp High ºF,Temp Avg ºF,Temp Low ºF,Dew High ºF,Dew Avg ºF,Dew Low ºF,Humid High %,Humid Avg %,Humid Low %,Speed High mph,Speed Avg mph,Speed Low mph,Pressure High in,Pressure Low in,Precip Sum in,city_code
2,2024-01-01,49.3,39.5,33.3,40.8,37.4,32.0,99,93,66,7.6,1.2,0.0,30.22,30.09,0.00,ICAMAR23
3,2024-01-02,44.6,39.5,32.9,44.2,38.5,32.4,99,97,86,5.4,0.8,0.0,30.23,30.12,0.00,ICAMAR23
4,2024-01-03,54.9,50.0,44.2,53.6,49.5,43.9,99,98,95,13.9,3.2,0.0,30.15,30.06,0.21,ICAMAR23
5,2024-01-04,50.9,48.1,44.6,50.4,47.7,44.2,99,99,95,6.7,0.9,0.0,30.14,29.84,0.22,ICAMAR23
6,2024-01-05,47.8,44.4,39.6,46.8,38.7,32.0,99,81,57,22.1,4.2,0.0,29.95,29.76,0.03,ICAMAR23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28,2024-07-27,100.0,84.5,68.9,68.4,64.1,59.5,84,53,27,10.7,2.1,0.0,29.90,29.77,0.00,IVILLA660
29,2024-07-28,103.3,86.8,68.4,67.3,61.7,55.6,82,46,22,10.7,1.6,0.0,29.93,29.81,0.00,IVILLA660
30,2024-07-29,96.6,85.9,73.9,71.8,62.2,52.2,72,47,29,25.1,2.9,0.0,30.00,29.79,0.00,IVILLA660
31,2024-07-30,102.0,86.5,71.2,71.4,66.0,55.9,89,53,29,10.5,1.9,0.0,29.95,29.84,0.00,IVILLA660


### 2.5 Save the information in a new file

In [12]:
df_final.to_csv("../data/historical_weather.csv")